In [1]:
import json
import nltk
import string
import re

from unidecode import unidecode

In [2]:
pontuacoes = string.punctuation + '…' + '‡' + '·' + '–' + 'º' + '§'
stopwords_portugues = nltk.corpus.stopwords.words('portuguese')
stopwords_portugues = stopwords_portugues + ["é", "ser", "ter", "dá", "art"]

numerais_romanos_pattern = re.compile('(^(?=[mdclxvi])M*(c[md]|d?c{0,3})(x[cl]|l?x{0,3})(i[xv]|v?i{0,3})$)')

In [3]:
def read_json(filename):
    with open(filename) as f:
        return json.load(f)

def dump_json(filename, json_object):
    with open(filename, "w") as fp:
        json.dump(json_object, fp) 

In [4]:
filename = "normas_80assuntos"
normas = read_json(f"{filename}.json")

In [5]:
def pre_processa_texto(text):
    # Lower
    text = text.lower()
    # Tokenização:
    tokens = nltk.word_tokenize(text)
    # Remoção de stopwords:
    tokens = [t for t in tokens if t not in stopwords_portugues]
    # Remoção de números:
    tokens = [re.sub(r'\d+', '', t) for t in tokens]
    # Remoção de pontuações:
    tokens = [re.sub('['+pontuacoes+']','', t) for t in tokens]
    # Remoção de números romanos:
    tokens = [numerais_romanos_pattern.sub('', t) for t in tokens]
    # Remoção de vazios:
    tokens = [t for t in tokens if t and len(t) > 1] 
    # Remoção de acentos:
    tokens = [unidecode(t) for t in tokens]
    return ' '.join(tokens)

def pre_processa_textos(normas, debug=False):
    for i, norma in enumerate(normas):
        if debug and i % 500 == 0:
            print(f'Processando norma {i}')
        norma['TextoPreProcessado'] = pre_processa_texto(norma['Texto'])

In [6]:
pre_processa_textos(normas, debug=True)

Processando norma 0
Processando norma 500
Processando norma 1000
Processando norma 1500
Processando norma 2000
Processando norma 2500
Processando norma 3000
Processando norma 3500
Processando norma 4000
Processando norma 4500
Processando norma 5000
Processando norma 5500
Processando norma 6000
Processando norma 6500
Processando norma 7000
Processando norma 7500
Processando norma 8000
Processando norma 8500
Processando norma 9000
Processando norma 9500


In [7]:
# Mantém apenas campos 'TextoPreProcessado', 'AssuntoGeral', 'Norma' 
normas = [{key: norma[key] for key in norma if key in ['TextoPreProcessado', 'AssuntoGeral', 'Norma']} for norma in normas]

In [11]:
dump_json(f"{filename}_processadas.json", normas);